In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from skimage.feature import peak_local_max
import cv2
from os import listdir
from os.path import isfile, join
import json
import pandas as pd
import time

In [2]:
def generate_df(filename):
    data = np.genfromtxt(filename, delimiter='\t')
    x_axis = data[0][1:-1]
    y_axis = np.flip(data[:,0][1:])
    data = np.delete(data, 818, axis=1)
    data = np.delete(data, 0, axis=0)
    data = np.delete(data, 0 ,axis=1)
    data = np.rot90(data, 3)
    data = data*(-1)
#     print("Data shape: ", data.shape)
    return pd.DataFrame(data=data, index=y_axis, columns=x_axis)

In [3]:
def convert_coordinates_to_frame(mask, frame, coordinates):
    vortices_list = []
    for x,y in coordinates:
        cx= int(x * frame.shape[0] / mask.shape[0])
        cy= int(y * frame.shape[0] / mask.shape[0])
        vortices_list.append((cx, cy))
    return vortices_list

# EXTRACTION MOMENTS OF TIME FOR WHICH THE DATA IS LABELED

In [4]:
''' EXTRACT TIME RANGE '''
mypath = 'CalculateDetectionMetrics/cropped_data_labeled/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
time_range = np.array(sorted([float(x[19:-5]) for x in list(filter(lambda s: 'json' in s, onlyfiles))]))
time_range = time_range[time_range >= 30]
time_range = np.round(time_range, 1)
time_range

array([ 30. ,  33. ,  36. ,  39. ,  42.1,  45. ,  48. ,  51. ,  54. ,
        57. ,  60. ,  63. ,  66. ,  69. ,  72. ,  75. ,  78. ,  81. ,
        84. ,  87. ,  90. ,  93. ,  96. ,  99. , 102. , 105. , 108. ,
       111. , 114. , 117. , 120. , 123. , 126. , 129. , 132. , 135. ,
       138. , 141. , 144. , 147. , 150. , 153. , 156. , 159. , 162. ,
       165. , 168. , 171. , 174. , 177. , 180. , 183. , 186. , 189. ,
       192. , 195. , 198. , 201. , 204. , 207. , 210.1, 214. , 217. ,
       220. , 223. , 226. , 229. , 232. , 235. , 238. , 241. , 244. ,
       247. , 250. , 253. , 256. , 259. , 262. , 265. , 268. , 271. ,
       274. , 277. , 280. , 283. , 286. , 289. , 292. , 295. , 298. ,
       301. , 304. , 307. , 310. , 313. , 316. , 319. , 322. , 325. ,
       328. ])

# CV BOHMIAN POTENTIAL METHOD ALGORITHM

In [6]:
program_starts = time.time()

detected_vortices = {}
for t in time_range:
    frame = cv2.imread(f'CalculateDetectionMetrics/cropped_data_labeled/x_vel_cropped_time={np.round(t, 1)}0.png')
    filename_bp = f'input_data/bohmian_potential//Bohmian_potential_time={np.round(t,1)}0.dat'
    df_bp = generate_df(filename_bp)
    
    ''' FIND LOCAL MINIMA IN BP DATA '''
    indices = peak_local_max(df_bp.values, threshold_abs=110, min_distance=5)
    
    '''GET DETECTED VORTICES COORDINATES'''
    detected_vortices[np.round(t,1)] = convert_coordinates_to_frame(df_bp.values, frame, coordinates=indices)
    
now = time.time()
print("It has been {0} seconds since the loop started".format(now - program_starts))

It has been 50.00749754905701 seconds since the loop started


# TP, FP, FN METRICS CALCULATIONS

In [7]:
''' READ JSON FILES WITH LABELED BOUNDING BOXES '''
bbox_size = 3
for index, t in enumerate(time_range):
    with open(f'CalculateDetectionMetrics/cropped_data_labeled/x_vel_cropped_time={np.round(t,1)}0.json', 'r') as f:
        dictData = json.load(f)
    frame = cv2.imread(f'CalculateDetectionMetrics/cropped_data_labeled/x_vel_cropped_time={np.round(t,1)}0.png')
    TP = []
    for i, vortex in enumerate(dictData['shapes']):
        labeled_mask = np.zeros((frame.shape[:2]), dtype='uint8')
        a = int(vortex['points'][0][0])
        b = int(vortex['points'][0][1])
        c = int(vortex['points'][1][0])
        d = int(vortex['points'][1][1])
    #         print(f'vortex {i+1}', a, b, c, d)
        cv2.rectangle(labeled_mask, (a,b), (c,d), 255, cv2.FILLED)
        for x,y in detected_vortices[np.round(t,1)]:
            if (x,y) not in TP:
                detected_vortex = np.zeros((frame.shape[:2]), dtype='uint8')
                cv2.rectangle(detected_vortex, (x-bbox_size,y-bbox_size), (x+bbox_size,y+bbox_size), 255, cv2.FILLED)
                result = np.all(cv2.bitwise_and(detected_vortex, labeled_mask) == 0)
                if not result:
                    TP.append((x,y))
                    break
    tp_count = len(TP) #CORRECT
    fp_count = len(detected_vortices[np.round(t,1)]) - len(TP) #INCORRECT
    fn_count = len(dictData['shapes']) - len(TP) #UNDETECTED
    
    ''' INSPECTION IF CALCULATION WORK PROPERLY '''
    if fp_count > 0:
        for (x,y) in detected_vortices[np.round(t,1)]:
            if (x,y) in TP:
                color = (0,255,0)
            else:
                color = (0,0,255)
            cv2.rectangle(frame, (x-12,y-12), (x+12,y+12), color, 2)
        for parameter, value, shift in zip(['T=', 'TP=', 'FP=', 'FN='], [t, tp_count, fp_count, fn_count], [30, 60, 90, 120]):
            cv2.putText(frame, text=parameter+str(value), org=(10, shift),
                    fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.9, color=(0,0,0),
                    thickness=1, lineType=cv2.LINE_AA)
        cv2.imwrite(f'BP_Method-Find_Local_Minima_in_2D_Array_RESULTS/TP_FP_{np.round(t,1)}.png', frame)
        print(t)
        
    ''' SAVE RESULTS TO FILE '''
#     print(t, tp_count, fp_count, fn_count)
    with open('BP_Method-Find_Local_Minima_in_2D_Array_RESULTS/BPMethod-Detection-Metrics-Results.csv', 'a') as fr:
        fr.write(f'{t}\t{tp_count}\t{fp_count}\t{fn_count}\n')

30.0
39.0
48.0
57.0
63.0
66.0
75.0
81.0
102.0
105.0
117.0
126.0
132.0
135.0
138.0
141.0
144.0
150.0
153.0
156.0
159.0
162.0
165.0
168.0
171.0
174.0
177.0
180.0
183.0
186.0
189.0
192.0
198.0
201.0
204.0
207.0
210.1
214.0
217.0
220.0
223.0
226.0
229.0
235.0
238.0
241.0
244.0
247.0
256.0
262.0
265.0
268.0
271.0
274.0
277.0
286.0
289.0
292.0
295.0
298.0
304.0
307.0
310.0
313.0
328.0


# FINAL METRICS SUMMATION

In [36]:
df_metrics = pd.read_csv('BP_Method-Find_Local_Minima_in_2D_Array_RESULTS/BPMethod-Detection-Metrics-Results.csv', header=None, delimiter='\t')
df_metrics.columns = ['TIME', 'TP', 'FP', 'FN']
df_metrics.set_index('TIME', inplace=True)
df_metrics.sum()

TP    2269
FP     291
FN      84
dtype: int64

# SAVE TO FILE DETECTED COORDINATES

In [9]:
with open('BP_Method-Find_Local_Minima_in_2D_Array_RESULTS/detected_vortices.txt','w') as data: 
      data.write(str(detected_vortices))

# GRID SEARCH OF PARAMETERS

In [5]:
distances = [1,2,3,4,5,6]
thresholds = [100, 110, 120, 130, 140, 150, 160, 170, 180]

for dist in distances:
    for thres in thresholds:
        detected_vortices = {}
        for t in time_range:
            frame = cv2.imread(f'CalculateDetectionMetrics/cropped_data_labeled/x_vel_cropped_time={np.round(t, 1)}0.png')
            filename_bp = f'input_data/bohmian_potential//Bohmian_potential_time={np.round(t,1)}0.dat'
            df_bp = generate_df(filename_bp)

            ''' FIND LOCAL MINIMA IN BP DATA '''
            indices = peak_local_max(df_bp.values, threshold_abs=thres, min_distance=dist)

            '''GET DETECTED VORTICES COORDINATES'''
            detected_vortices[np.round(t,1)] = convert_coordinates_to_frame(df_bp.values, frame, coordinates=indices)
    
    
    
    
        ''' READ JSON FILES WITH LABELED BOUNDING BOXES '''
        bbox_size = 3
        for index, t in enumerate(time_range):
            with open(f'CalculateDetectionMetrics/cropped_data_labeled/x_vel_cropped_time={np.round(t,1)}0.json', 'r') as f:
                dictData = json.load(f)
            frame = cv2.imread(f'CalculateDetectionMetrics/cropped_data_labeled/x_vel_cropped_time={np.round(t,1)}0.png')
            TP = []
            for i, vortex in enumerate(dictData['shapes']):
                labeled_mask = np.zeros((frame.shape[:2]), dtype='uint8')
                a = int(vortex['points'][0][0])
                b = int(vortex['points'][0][1])
                c = int(vortex['points'][1][0])
                d = int(vortex['points'][1][1])
            #         print(f'vortex {i+1}', a, b, c, d)
                cv2.rectangle(labeled_mask, (a,b), (c,d), 255, cv2.FILLED)
                for x,y in detected_vortices[np.round(t,1)]:
                    if (x,y) not in TP:
                        detected_vortex = np.zeros((frame.shape[:2]), dtype='uint8')
                        cv2.rectangle(detected_vortex, (x-bbox_size,y-bbox_size), (x+bbox_size,y+bbox_size), 255, cv2.FILLED)
                        result = np.all(cv2.bitwise_and(detected_vortex, labeled_mask) == 0)
                        if not result:
                            TP.append((x,y))
                            break
            tp_count = len(TP) #CORRECT
            fp_count = len(detected_vortices[np.round(t,1)]) - len(TP) #INCORRECT
            fn_count = len(dictData['shapes']) - len(TP) #UNDETECTED


            ''' SAVE RESULTS TO FILE '''
        #     print(t, tp_count, fp_count, fn_count)
            with open(f'BP_Method-Find_Local_Minima_in_2D_Array_RESULTS/EXPERIMENT_WITH_GRIDSEARCH_THRESH={thres}_DIST={dist}.csv', 'a') as fr:
                fr.write(f'{t}\t{tp_count}\t{fp_count}\t{fn_count}\n')